# Population and employment dynamics in states and counties of the USA

In [ ]:
import ipywidgets as widgets
import ipyleaflet as leaflet
import matplotlib.pyplot as plt
import json

from popemp import data
from popemp.analysis import data_yr, data_ag, color_from_gr

geo = data.geo()

GeoData is a natural choise for area layer, but it does not support `style_callback` ([GH PR](https://github.com/jupyter-widgets/ipyleaflet/pull/786)). Using GeoJSON instead.

In [ ]:
def data_map(st, y0, y1, abs_rel):
    if st == '00':
        df = geo.query('cty == "000"')
    else:
        df = geo.query('st == @st')

    df = df.merge(data_ag(y0, y1))
    df['color'] = color_from_gr(df, abs_rel)
    df = df[['st', 'cty', 'name', 'geometry', 'color']]
    return df

def click_geo(**kw):
    p = kw['properties']
    y0, y1 = year_selector.value
    upd_graph(p['st'], p['cty'], y0, y1)

def area_style(feature):
    style = dict(fillColor=feature['properties']['color'])
    return style
        
def refresh(_):
    s = state_selector.value
    y0, y1 = year_selector.value
    if len(map_.layers) == 2:
        map_.remove_layer(map_.layers[1])
    geo_json = json.loads(data_map(s, y0, y1, abs_rel_selector.value).to_json())
    l = leaflet.GeoJSON(data=geo_json,
                        style={'stroke': False, 'fillOpacity': 0.5},
                        hover_style={'stroke': True},
                        style_callback=area_style)
    l.on_click(click_geo)
    map_.add_layer(l)
    upd_graph(s, '000', y0, y1)

def upd_graph(st, cty, y0, y1):
    with graph:
        graph.clear_output(True)
        fig, ax = plt.subplots(3)
        plt.close()
        data_yr.query('st == @st and cty == @cty and year >= @y0 and year <= @y1').set_index('year')['pop'].plot(ax=ax[0])
        data_yr.query('st == @st and cty == @cty and year >= @y0 and year <= @y1').set_index('year')['emp'].plot(ax=ax[1])
        data_yr.query('st == @st and cty == @cty and year >= @y0 and year <= @y1').set_index('year')[['pop_gr', 'emp_gr']].plot(ax=ax[2])
        fig.suptitle(st)
        display(fig)

In [ ]:
year_selector = widgets.IntRangeSlider(value=(2007, 2012), min=1991, max=2019)
abs_rel_selector = widgets.RadioButtons(options=[('Absolute', 'abs'), ('Relative', 'rel')])
state_selector = widgets.Dropdown(options=data_yr['st'].unique())
refresh_button = widgets.Button(description='Refresh')
refresh_button.on_click(refresh)
controls = widgets.VBox([year_selector, abs_rel_selector, widgets.Label('Select US or state'), state_selector, refresh_button])
map_ = leaflet.Map(center=(40, -95), zoom=4)
graph = widgets.Output()

refresh(None)
upd_graph('00', '000', *year_selector.value)

widgets.AppLayout(left_sidebar=controls,
                center=map_, 
                right_sidebar=graph)